In [ ]:
import warnings 
warnings.filterwarnings("ignore")

Welcome to the world where fashion meets computer vision! 

# **FASHION COLLABORATING WITH COMPUTER VISION**

Here, we are importing the basic library functions which are necessary to carry on the operations.

In [ ]:
import os
import gc
import sys
import math
import json
import glob
import random
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import skimage.io
from IPython.display import clear_output

import itertools
from tqdm import tqdm

from imgaug import augmenters as iaa
from sklearn.model_selection import StratifiedKFold, KFold

import tensorflow as tf

The data required for this project is in the Google Drive. We are mounting on to COLAB in this step.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In the next step, we are specifying path to few variables.

In [ ]:
TRAIN_IMAGE_DIR = Path('/content/drive/My Drive/stuff/train')
DATA_DIR = Path('/content/drive/My Drive')
ROOT_DIR = Path('/content/drive/My Drive')

#**Data Import**
Importing labels file through pandas library

In [ ]:
# import train file 
import pandas as pd
train = pd.read_csv(str(DATA_DIR/'train.csv'))
train.head()
# train = train[train['ClassId'] <= "10"]
# train.head()


In [ ]:
print(len(train))

 extracting image metadata fom json file

In [ ]:
 
with open(str(DATA_DIR/'Label.json')) as f:
    label_descriptions = json.load(f)

label_names = [x['name'] for x in label_descriptions['categories']]
label_names

In [ ]:
label_df = pd.DataFrame(label_names).reset_index()
label_df.columns = ['Id','Labels']
label_df.head()

In [ ]:
print(len(label_df))

In [ ]:
#ClassId = 0,1 and 10

segment_df = train
segment_df['CategoryId'] = segment_df['ClassId'].str.split('_').str[0]


#df2 = []
"""
for i in range(47):
  df1 = segment_df[segment_df['CategoryId'] == i]
  #print(df1)
  df2 = df2.append(df1)
  #print(df2)
  i += 1
  """

df0 = segment_df[segment_df['CategoryId'] == "0"]
df1 = segment_df[segment_df['CategoryId'] == "1"] 
df2 = segment_df[segment_df['CategoryId'] == "2"]
df3 = segment_df[segment_df['CategoryId'] == "3"]
df4 = segment_df[segment_df['CategoryId'] == "4"]
df5 = segment_df[segment_df['CategoryId'] == "5"]

df6 = segment_df[segment_df['CategoryId'] == "6"]
df7 = segment_df[segment_df['CategoryId'] == "7"] 
df8 = segment_df[segment_df['CategoryId'] == "8"]
df9 = segment_df[segment_df['CategoryId'] == "9"]
df10 = segment_df[segment_df['CategoryId'] == "10"]
df11 = segment_df[segment_df['CategoryId'] == "11"]

df12 = segment_df[segment_df['CategoryId'] == "12"]
df13 = segment_df[segment_df['CategoryId'] == "13"] 
df14 = segment_df[segment_df['CategoryId'] == "14"]
df15 = segment_df[segment_df['CategoryId'] == "15"]
df16 = segment_df[segment_df['CategoryId'] == "16"]
df17 = segment_df[segment_df['CategoryId'] == "17"]

df18 = segment_df[segment_df['CategoryId'] == "18"]
df19 = segment_df[segment_df['CategoryId'] == "19"] 
df20 = segment_df[segment_df['CategoryId'] == "20"]
df21 = segment_df[segment_df['CategoryId'] == "21"]
df22 = segment_df[segment_df['CategoryId'] == "22"]
df23 = segment_df[segment_df['CategoryId'] == "23"]

df24 = segment_df[segment_df['CategoryId'] == "24"]
df25 = segment_df[segment_df['CategoryId'] == "25"] 
df26 = segment_df[segment_df['CategoryId'] == "26"]
df27 = segment_df[segment_df['CategoryId'] == "27"]
df28 = segment_df[segment_df['CategoryId'] == "28"]
df29 = segment_df[segment_df['CategoryId'] == "29"]



#segment_df = segment_df[:10]
df1 = df1.append(df0)
df1 = df1.append(df1)
df1 = df1.append(df2)
df1 = df1.append(df3)
df1 = df1.append(df4)
df1 = df1.append(df5)

df1 = df1.append(df6)
df1 = df1.append(df7)
df1 = df1.append(df8)
df1 = df1.append(df9)
df1 = df1.append(df10)
df1 = df1.append(df11)

df1 = df1.append(df12)
df1 = df1.append(df13)
df1 = df1.append(df14)
df1 = df1.append(df15)
df1 = df1.append(df16)
df1 = df1.append(df17)

df1 = df1.append(df18)
df1 = df1.append(df19)
df1 = df1.append(df20)
df1 = df1.append(df21)
df1 = df1.append(df22)
df1 = df1.append(df23)

df1 = df1.append(df24)
df1 = df1.append(df25)
df1 = df1.append(df26)
df1 = df1.append(df27)
df1 = df1.append(df28)
df1 = df1.append(df29)



segment_df = df1
print("Total segments: ", len(segment_df))
segment_df.head()

In [ ]:
# segment_df = train
# segment_df['CategoryId'] = segment_df['ClassId'].str.split('_').str[0]
# # segment_df = segment_df[segment_df['ClassId'] <= "10"]
# # train.head()
# print("Total segments: ", len(segment_df))
# segment_df


Rows with the same image are grouped together because the subsequent operations perform at an image level

In [ ]:

image_df = segment_df.groupby('ImageId')['EncodedPixels', 'CategoryId'].agg(lambda x: list(x))
size_df = segment_df.groupby('ImageId')['Height', 'Width'].mean()
image_df = image_df.join(size_df, on='ImageId')

print("Total images: ", len(image_df))
image_df

In [ ]:
# number of labels per image
labels_per_image = image_df['CategoryId'].map(lambda x:len(x)).value_counts().to_frame().reset_index().sort_values(by = 'index')
labels_per_image.columns = ['#labels','#images']

plt.figure(figsize=(15, 7))
sns.barplot(labels_per_image['#labels'],labels_per_image['#images'])
plt.title("Number of Labels per Image", fontsize=20)
plt.xlabel("# of labels", fontsize=20)
plt.ylabel("# of images", fontsize=20)
plt.show()

In [ ]:
segment_df['CategoryId'] = segment_df['CategoryId'].astype('int64')
labels_per_image2 = segment_df.merge(label_df, how='left', left_on='CategoryId', right_on='Id')
labels_per_image3 = labels_per_image2.groupby('Labels')['ImageId'].nunique().to_frame().reset_index()
labels_per_image2.head()
#labels_per_image3.head()

In [ ]:
labels_per_image4 = labels_per_image2.groupby('Labels')['ImageId'].count().to_frame().reset_index()
labels_per_image4.head(4)

In [ ]:
labels_per_image4.to_csv('word_cloud_data.csv')

In [ ]:
d = {}
for i in range(len(labels_per_image4)):
    d[labels_per_image4.iloc[i,0]] = labels_per_image4.iloc[i,1]

In [ ]:
plt.figure(figsize=(20, 7))
sns.barplot(labels_per_image3['Labels'],labels_per_image3['ImageId'])
plt.xticks(rotation=90)
plt.title("Labels Distribution in Images", fontsize=20)
plt.xlabel("labels", fontsize=10)
plt.ylabel("# of images", fontsize=10)
plt.show()

As the data is bulk, we limited the data and labels which would help us in obtaintaing faster results.

In [ ]:
count = 1
coun = len(os.listdir(TRAIN_IMAGE_DIR))
x = []
while(count<=5000):
  for file in os.listdir(TRAIN_IMAGE_DIR):
    if (coun >= 1):
      images=file
      x.append(images)
      count+=1
      coun -= 1
      #print(coun)
print(len(x))
      

In [ ]:

#count = 0
#coun = len(os.listdir(TRAIN_IMAGE_DIR))
#images = [s for s in os.listdir(TRAIN_IMAGE_DIR) if (i <= 10000) else 0]
images = x
uploaded_images = pd.DataFrame(images, columns = ['image_name'])
image_df = image_df[image_df.index.isin(uploaded_images['image_name'])]



In [ ]:
image_df.shape

# **Partition data in train and test**

In [ ]:

FOLD = 0
N_FOLDS = 10

kf = KFold(n_splits=N_FOLDS, random_state=42, shuffle=True)
splits = kf.split(image_df) # ideally, this should be multilabel stratification

def get_fold():    
    for i, (train_index, valid_index) in enumerate(splits):
        if i == FOLD:
            return image_df.iloc[train_index], image_df.iloc[valid_index]
        
train_df, valid_df = get_fold()

Apply Mask R-CNN with COCO pretrained weights to the task

In [ ]:
!rm -rf Mask_RCNN

# **Mask-RCNN implementation**

In [ ]:

!git clone https://github.com/Kedar-V/Mask_RCNN.git
os.chdir('Mask_RCNN')

!rm -rf .git # to prevent an error when the kernel is committed
!rm -rf images assets # to prevent displaying images at the bottom of a kernel

sys.path.append(ROOT_DIR/'Mask_RCNN')
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = 'mask_rcnn_coco.h5'

#**CONFIGURATIONS**

Mask R-CNN has a load of hyperparameters. I only adjust some of them.

In [ ]:
# Set configuration

NUM_CATS = 30  # classification ignoring attributes (only categories)
IMAGE_SIZE = 512 # the image size is set to 512, which is the same as the size of submission masks

class FashionConfig(Config):
    NAME = "fashion"
    NUM_CLASSES = NUM_CATS + 1 # +1 for the background class
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4 #Batch size - memory error occurs when IMAGES_PER_GPU is too high
    #https://datascience.stackexchange.com/questions/29719/how-to-set-batch-size-steps-per-epoch-and-validation-steps

    #IMAGE_META_SIZE = 43
    
    BACKBONE = 'resnet50' #resnet50 will be lighter than resnet101 for training
    
    IMAGE_MIN_DIM = IMAGE_SIZE
    IMAGE_MAX_DIM = IMAGE_SIZE    
    IMAGE_RESIZE_MODE = "none"
    
#     RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    RPN_ANCHOR_SCALES = (4, 8, 16, 32, 64)
    #DETECTION_NMS_THRESHOLD = 0.7

    STEPS_PER_EPOCH = 50
    VALIDATION_STEPS = 100

    MAX_GT_INSTANCES = 2
    DETECTION_MAX_INSTANCES = 2
    ## balance out losses
    # https://stackoverflow.com/questions/55360262/what-exactly-are-the-losses-in-matterport-mask-r-cnn
    # https://stackoverflow.com/questions/46272841/what-is-the-loss-function-of-the-mask-rcnn
    LOSS_WEIGHTS = {
          "rpn_class_loss": 1.0, # How well the Region Proposal Network separates background with objetcs
          "rpn_bbox_loss": 0.8, # How well the RPN localize objects
          "mrcnn_class_loss": 6.0, # How well the Mask RCNN localize objects
          "mrcnn_bbox_loss": 6.0, # How well the Mask RCNN recognize each class of object
          "mrcnn_mask_loss": 6.0 # How well the Mask RCNN segment objects
    }
    
config = FashionConfig()
config.display()

Here is the custom function that resizes an image

In [ ]:
# resizing image to 512X512;
def resize_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA)  
    return img

In [ ]:
class FashionDataset(utils.Dataset):

    def __init__(self, df):
        super().__init__(self)
        
        # Add classes
#         for i, name in enumerate(label_names):
#             print(i, end = " ")
#             print(name)
        self.add_class("fashion", 0, "shirt, blouse")
        self.add_class("fashion", 1, "top, t-shirt, sweatshirt")
        self.add_class("fashion", 2, "sweater")
        self.add_class("fashion", 3, "cardigan")
        self.add_class("fashion", 4, "jacket")
        
        self.add_class("fashion", 5, "vest")
        self.add_class("fashion", 6, "pants")
        self.add_class("fashion", 7, "shorts")
        self.add_class("fashion", 8, "skirt")
        self.add_class("fashion", 9, "coat")
        self.add_class("fashion", 10, "dress")

        self.add_class("fashion", 11, "jumpsuit")
        self.add_class("fashion", 12, "cape")
        self.add_class("fashion", 13, "glasses")
        self.add_class("fashion", 14, "hat")
        self.add_class("fashion", 15, "headband, head covering, hair accessory")
        
        self.add_class("fashion", 16, "tie")
        self.add_class("fashion", 17, "glove")
        self.add_class("fashion", 18, "watch")
        self.add_class("fashion", 19, "belt")
        self.add_class("fashion", 20, "leg warmer")
        self.add_class("fashion", 21, "tights, stockings")

        
        self.add_class("fashion", 22, "sock")
        self.add_class("fashion", 23, "shoe")
        self.add_class("fashion", 24, "bag, wallet")
        self.add_class("fashion", 25, "scarf")
        self.add_class("fashion", 26, "umbrella")
        self.add_class("fashion", 27, "hood")
        self.add_class("fashion", 28, "collar")
        self.add_class("fashion", 29, "lapel")

        
        # Add images 
        for i, row in df.iterrows():
            self.add_image("fashion", 
                           image_id=row.name, 
                           path=str(TRAIN_IMAGE_DIR/row.name), 
                           labels=row['CategoryId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [label_names[int(x)] for x in info['labels']]
    
    def load_image(self, image_id):
        return resize_image(self.image_info[image_id]['path'])
    def load_mask(self, image_id):
        info = self.image_info[image_id]
                
        mask = np.zeros((IMAGE_SIZE, IMAGE_SIZE, len(info['annotations'])), dtype=np.uint8)
        labels = []
        
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(info['height']*info['width'], 0, dtype=np.uint8)
            annotation = [int(x) for x in annotation.split(' ')]
            
            for i, start_pixel in enumerate(annotation[::2]):
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1
            sub_mask = sub_mask.reshape((info['height'], info['width']), order='F')
            sub_mask = cv2.resize(sub_mask, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
            
            mask[:, :, m] = sub_mask
            labels.append(int(label)+1)
            
        return mask, np.array(labels)



#**VISUALIZATION**

In [ ]:
# Visualizing random images
dataset = FashionDataset(image_df)
dataset.prepare()

for i in range(1):
    image_id = random.choice(dataset.image_ids)
    print(dataset.image_reference(image_id))
    
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    print(mask)
    plt.imshow(image)
    #visualize.display_top_masks(image, mask, class_ids, dataset.class_names, limit=4)

# Prepare Data

In [ ]:

train_dataset = FashionDataset(train_df)
train_dataset.prepare()

valid_dataset = FashionDataset(valid_df)
valid_dataset.prepare()

# Image augmentation

In [ ]:

augmentation = iaa.Sequential([
    iaa.OneOf([ ## rotate
        iaa.Affine(rotate=0),
        iaa.Affine(rotate=90),
        iaa.Affine(rotate=180),
        iaa.Affine(rotate=270),
    ]),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen
        iaa.GaussianBlur(sigma=(0.0, 0.3)),
        iaa.Sharpen(alpha=(0.0, 0.3)),
    ]),
])

# sample output

In [ ]:

imggrid = augmentation.draw_grid(image, cols=5, rows=2)
plt.figure(figsize=(20, 10))
_ = plt.imshow(imggrid.astype(int))

# initiating Mask R-CNN training

In [ ]:
import tensorflow
print(tf.__version__)


In [ ]:
!pip install tensorflow==1.14.0

In [ ]:
!pip install keras==2.2.5

In [ ]:


model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR);
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    'mrcnn_class_logits', 'mrcnn_bbox_fc', 'mrcnn_bbox', 'mrcnn_mask'])

In [ ]:
# Declaring learning rate
LR = 0.03

In [ ]:
%%time
model.train(train_dataset, valid_dataset,
            learning_rate=LR/6,
            epochs=10,
            layers='all',
            augmentation=augmentation)

history = model.keras_model.history.history

In [ ]:
%%time
model.train(train_dataset, valid_dataset,
            learning_rate=LR/8,
            epochs=15,
            layers='all',
            augmentation=augmentation)

new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
epochs = range(1, len(history['loss'])+1)
pd.DataFrame(history, index=epochs)

In [ ]:
# find best epoch
best_epoch = np.argmin(history["val_loss"]) + 1
print("Best epoch: ", best_epoch)
print("Valid loss: ", history["val_loss"][best_epoch-1])

In [ ]:
#glob_list = glob.glob(f'/content/drive/My Drive/fashion20201208T0049/mask_rcnn_fashion_0001.h5')
#model_path = glob_list[0] if glob_list else ''


# Prediction, this cell defines InferenceConfig and loads the best trained model.

In [ ]:

class InferenceConfig(FashionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=ROOT_DIR)

model.load_weights(model_path, by_name=True)

we have to fix the overlappsed masks

In [ ]:

def refine_masks(masks, rois):
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    mask_index = np.argsort(areas)
    union_mask = np.zeros(masks.shape[:-1], dtype=bool)
    for m in mask_index:
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
        union_mask = np.logical_or(masks[:, :, m], union_mask)
    for m in range(masks.shape[-1]):
        mask_pos = np.where(masks[:, :, m]==True)
        if np.any(mask_pos):
            y1, x1 = np.min(mask_pos, axis=1)
            y2, x2 = np.max(mask_pos, axis=1)
            rois[m, :] = [y1, x1, y2, x2]
    return masks, rois

 Let’s load an image and try to see how the model performs. You can use any of your images to test the model.

In [ ]:

# Load a random image from the images folder
import skimage.io
image_path = '/content/drive/My Drive/f9c5f3efdae7ce33986891e0ba2f0a9c.jpg'

# original image
plt.figure(figsize=(12,10))
skimage.io.imshow(skimage.io.imread(image_path))

img = skimage.io.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

result = model.detect([resize_image(image_path)])
r = result[0]
if r['masks'].size > 0:
    masks = np.zeros((img.shape[0], img.shape[1], r['masks'].shape[-1]), dtype=np.uint8)
    for m in range(r['masks'].shape[-1]):
        masks[:, :, m] = cv2.resize(r['masks'][:, :, m].astype('uint8'), 
                                    (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    y_scale = img.shape[0]/IMAGE_SIZE
    x_scale = img.shape[1]/IMAGE_SIZE
    rois = (r['rois'] * [y_scale, x_scale, y_scale, x_scale]).astype(int)
    
    masks, rois = refine_masks(masks, rois)
else:
    masks, rois = r['masks'], r['rois']

visualize.display_instances(img, rois, masks, r['class_ids'], 
                            ['bg']+label_names, r['scores'])
    